this notebook make use of 200 samples, train and test them to see if the test result is good.
further, it adds a penalty for negative predictions in the custom MSE loss fn

In [8]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [18]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import math
import matplotlib.pyplot as plt
from keras.utils import plot_model

In [3]:
!python3 --version

Python 3.8.10


In [107]:
dataset = np.array([16733227350748467, 16733227550391883, 16733227750050025, 16733227949706609, 16733228149363639, 16733228349019607, 16733228548677527, 16733228748334981, 16733228947993627, 16733229147650111, 16733229347309299, 16733229546967197, 16733229746624267, 16733229946280917, 16733230145942315, 16733230345601135, 16733230545259547, 16733230744916627, 16733230944574515, 16733231144231219, 16733231343887957, 16733231543544763, 16733231743201507, 16733231942858347, 16733232142523681, 16733232342180973, 16733232541839835, 16733232741504825, 16733232941162645, 16733233140821019, 16733233340496189, 16733233540153285, 16733233739810287, 16733233939477109, 16733234139133039, 16733234338789617, 16733234538455601, 16733234738115653, 16733234937778513, 16733235137451875, 16733235337110475, 16733235536769513, 16733235736443939, 16733235936099919, 16733236135758019, 16733236335433487, 16733236535089911, 16733236734750133, 16733236934424299, 16733237134081295, 16733237333763445, 16733237533427401, 16733237733084659, 16733237932741693, 16733238132399255, 16733238332054073, 16733238531713055, 16733238731368065, 16733238931026683, 16733239130682383, 16733239330337993, 16733239529995019, 16733239729653093, 16733239929308407, 16733240128965159, 16733240328622337, 16733240528278847, 16733240727937145, 16733240927593905, 16733241127249951, 16733241326908781, 16733241526565401, 16733241726223181, 16733241925879327, 16733242125537299, 16733242325193887, 16733242524850679, 16733242724506711, 16733242924164657, 16733243123821195, 16733243323478221, 16733243523136857, 16733243722793281, 16733243922450205, 16733244122116275, 16733244321772539, 16733244521429299, 16733244721097333, 16733244920753177, 16733245120409325, 16733245320076593, 16733245519735605, 16733245719392665, 16733245919067407, 16733246118725775, 16733246318380823, 16733246518053371, 16733246717710363, 16733246917369085, 16733247117042779, 16733247316701879, 16733247516359167, 16733247716034493, 16733247915691399, 16733248115349863, 16733248315020933, 16733248514680855, 16733248714338391, 16733248914013313, 16733249113670167, 16733249313344273, 16733249513012467, 16733249712668399, 16733249912326273, 16733250111981763, 16733250311636913, 16733250511292945, 16733250710950579, 16733250910608017, 16733251110263771, 16733251309920797, 16733251509576173, 16733251709232571, 16733251908888987, 16733252108545827, 16733252308202529, 16733252507859085, 16733252707517577, 16733252907174435, 16733253106831785, 16733253306488551, 16733253506144059, 16733253705800105, 16733253905457737, 16733254105117091, 16733254304772883, 16733254504429251, 16733254704086333, 16733254903742893, 16733255103402383, 16733255303060171, 16733255502718759, 16733255702375687, 16733255902050431, 16733256101708513, 16733256301364273, 16733256501030351, 16733256700686273, 16733256900343759, 16733257100010439, 16733257299667619, 16733257499324323, 16733257698997745, 16733257898654383, 16733258098310867, 16733258297976265, 16733258497634191, 16733258697291101, 16733258896966919, 16733259096622879, 16733259296279309, 16733259495953755, 16733259695610383, 16733259895266821, 16733260094939707, 16733260294596831, 16733260494254455, 16733260693930141, 16733260893586709, 16733261093243467, 16733261292921355, 16733261492579391, 16733261692236647, 16733261891894235, 16733262091551005, 16733262291207325, 16733262490864983, 16733262690521773, 16733262890179571, 16733263089834991, 16733263289494231, 16733263489148607, 16733263688806325, 16733263888463159, 16733264088118731, 16733264287775607, 16733264487432673, 16733264687089773, 16733264886747791, 16733265086404485, 16733265286060991, 16733265485719071, 16733265685374719, 16733265885030443, 16733266084688573, 16733266284345761, 16733266484002783, 16733266683659769, 16733266883317777, 16733267082975031])
dataset = dataset * 0.00000001

In [108]:
dataset[0:2]

array([1.67332274e+08, 1.67332276e+08])

In [109]:
dataset.shape

(200,)

In [110]:
start = dataset[0]
end = dataset[-1]
interval = end - start
start, end, interval


(167332273.50748467, 167332670.82975033, 397.3222656548023)

In [111]:
dataset = dataset - start
dataset[0], dataset[-1]

(0.0, 397.3222656548023)

In [112]:
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = dataset.reshape(-1,1)
dataset = scaler.fit_transform(dataset)
dataset

array([[0.        ],
       [0.00502472],
       [0.01004982],
       [0.01507487],
       [0.02009994],
       [0.02512497],
       [0.03015006],
       [0.03517514],
       [0.04020024],
       [0.04522529],
       [0.05025041],
       [0.0552755 ],
       [0.06030057],
       [0.06532562],
       [0.0703508 ],
       [0.07537591],
       [0.08040101],
       [0.08542607],
       [0.09045116],
       [0.09547622],
       [0.10050128],
       [0.10552634],
       [0.11055139],
       [0.11557645],
       [0.12060173],
       [0.1256268 ],
       [0.13065191],
       [0.13567718],
       [0.14070226],
       [0.14572736],
       [0.15075288],
       [0.15577795],
       [0.16080301],
       [0.16582833],
       [0.17085336],
       [0.17587842],
       [0.18090371],
       [0.18592885],
       [0.19095406],
       [0.19597954],
       [0.20100464],
       [0.20602976],
       [0.21105526],
       [0.2160803 ],
       [0.22110539],
       [0.22613092],
       [0.23115597],
       [0.236

In [113]:
dataset.shape

(200, 1)

In [114]:
old_dataset = scaler.inverse_transform(dataset)
old_dataset

array([[  0.        ],
       [  1.99643418],
       [  3.99301556],
       [  5.98958141],
       [  7.98615173],
       [  9.9827114 ],
       [ 11.9792906 ],
       [ 13.97586513],
       [ 15.9724516 ],
       [ 17.96901646],
       [ 19.96560833],
       [ 21.96218729],
       [ 23.958758  ],
       [ 25.9553245 ],
       [ 27.95193848],
       [ 29.94852668],
       [ 31.9451108 ],
       [ 33.94168162],
       [ 35.9382605 ],
       [ 37.93482754],
       [ 39.93139488],
       [ 41.92796296],
       [ 43.92453042],
       [ 45.9210988 ],
       [ 47.91775212],
       [ 49.91432506],
       [ 51.91091368],
       [ 53.90756357],
       [ 55.90414178],
       [ 57.90072554],
       [ 59.89747721],
       [ 61.89404815],
       [ 63.89061821],
       [ 65.88728639],
       [ 67.88384572],
       [ 69.88041148],
       [ 71.87707132],
       [ 73.87367186],
       [ 75.87030044],
       [ 77.86703408],
       [ 79.8636201 ],
       [ 81.86021045],
       [ 83.85695472],
       [ 85

In [115]:
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
train_size,test_size

(134, 66)

In [116]:
look_back = 50
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [117]:
trainX, trainY = create_dataset(train, look_back=look_back)
testX, testY = create_dataset(test, look_back)

In [118]:
trainX[:2],trainY[:2]
testX[:2],testY[:2]

(array([[0.67336696, 0.67839199, 0.68341704, 0.68844211, 0.69346716,
         0.69849229, 0.70351737, 0.70854248, 0.71356754, 0.71859305,
         0.72361814, 0.72864318, 0.73366847, 0.73869351, 0.74371858,
         0.74874389, 0.75376896, 0.75879402, 0.7638195 , 0.76884455,
         0.7738696 , 0.77889488, 0.78391997, 0.78894503, 0.79397057,
         0.79899561, 0.80402066, 0.80904616, 0.81407121, 0.81909626,
         0.82412173, 0.8291468 , 0.83417188, 0.83919741, 0.84422246,
         0.84924752, 0.85427311, 0.8592982 , 0.86432327, 0.86934835,
         0.87437341, 0.87939846, 0.88442354, 0.8894486 , 0.89447368,
         0.89949871, 0.90452383, 0.90954883, 0.91457391, 0.91959897],
        [0.67839199, 0.68341704, 0.68844211, 0.69346716, 0.69849229,
         0.70351737, 0.70854248, 0.71356754, 0.71859305, 0.72361814,
         0.72864318, 0.73366847, 0.73869351, 0.74371858, 0.74874389,
         0.75376896, 0.75879402, 0.7638195 , 0.76884455, 0.7738696 ,
         0.77889488, 0.78391997, 

In [119]:
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [120]:
trainX.shape

(83, 1, 50)

In [121]:
trainY

array([0.25125738, 0.25628262, 0.26130769, 0.26633275, 0.27135783,
       0.27638284, 0.28140795, 0.28643297, 0.29145807, 0.29648311,
       0.30150813, 0.3065332 , 0.31155829, 0.31658331, 0.32160837,
       0.32663344, 0.33165849, 0.33668359, 0.34170865, 0.34673369,
       0.3517588 , 0.35678385, 0.36180894, 0.36683398, 0.37185907,
       0.37688412, 0.38190918, 0.38693422, 0.39195931, 0.39698436,
       0.40200943, 0.40703454, 0.41205959, 0.41708465, 0.42210994,
       0.42713499, 0.43216005, 0.43718539, 0.44221042, 0.44723547,
       0.45226079, 0.4572859 , 0.46231097, 0.46733648, 0.47236158,
       0.4773866 , 0.48241205, 0.48743712, 0.49246222, 0.49748771,
       0.50251282, 0.5075379 , 0.51256342, 0.51758848, 0.52261359,
       0.527639  , 0.53266414, 0.53768922, 0.54271474, 0.5477398 ,
       0.55276529, 0.55779064, 0.56281568, 0.56784076, 0.57286579,
       0.57789081, 0.58291585, 0.58794093, 0.592966  , 0.59799104,
       0.6030161 , 0.60804112, 0.61306617, 0.61809122, 0.62311

In [122]:
import keras.backend as K

def custom_loss(y_true, y_pred):
    # Calculate the squared error between true and predicted values
    squared_error = K.square(y_true - y_pred)
    
    # Define a penalty factor for negative predictions
    penalty_factor = 1000000
    
    # Apply the penalty factor to negative predictions
    penalized_error = K.switch(y_pred < 0, squared_error * penalty_factor, squared_error)
    
    # Calculate the mean of the penalized errors
    loss = K.mean(penalized_error)
    
    return loss

In [19]:
look_back = 50
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))


# Visualize the Model Architecture
plot_model(model, to_file='model_visualization.png', show_shapes=True)
# model.compile(loss=custom_loss, optimizer='adam', metrics=['accuracy'])
# model.fit(trainX, trainY, epochs=10, batch_size=1, verbose=2)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [11]:
inference_model = tf.keras.models.clone_model(model)
inference_model

In [125]:
import time
start_time = time.perf_counter()
# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

end_time = time.perf_counter()
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

start_time, end_time, end_time - start_time

1/1 [==============================] - 0s 20ms/step
Train Score: 4.25 RMSE
Test Score: 59.91 RMSE


(18903.027129315, 18903.46708271, 0.439953395001794)

In [126]:
trainPredict, trainY

(array([[ 89.80791 ],
        [ 92.35823 ],
        [ 94.90679 ],
        [ 97.452614],
        [ 99.994644],
        [102.53201 ],
        [105.06372 ],
        [107.58894 ],
        [110.10676 ],
        [112.616455],
        [115.11718 ],
        [117.608185],
        [120.088844],
        [122.55847 ],
        [125.01643 ],
        [127.46212 ],
        [129.895   ],
        [132.31458 ],
        [134.7203  ],
        [137.11182 ],
        [139.48856 ],
        [141.8503  ],
        [144.19656 ],
        [146.52707 ],
        [148.8415  ],
        [151.13963 ],
        [153.42119 ],
        [155.6859  ],
        [157.93367 ],
        [160.16426 ],
        [162.37749 ],
        [164.5733  ],
        [166.75157 ],
        [168.9122  ],
        [171.05513 ],
        [173.18033 ],
        [175.28772 ],
        [177.37737 ],
        [179.44919 ],
        [181.50327 ],
        [183.53958 ],
        [185.5582  ],
        [187.5592  ],
        [189.54262 ],
        [191.50853 ],
        [1

In [127]:
testPredict,testY

(array([[315.0473 ],
        [316.01138],
        [316.96933],
        [317.92126],
        [318.8671 ],
        [319.8071 ],
        [320.74124],
        [321.66953],
        [322.5921 ],
        [323.50888],
        [324.42   ],
        [325.32556],
        [326.22546],
        [327.1199 ],
        [328.00888]], dtype=float32),
 array([[367.37370265, 369.37027141, 371.36684206, 373.36341304,
         375.35999325, 377.35656017, 379.35312524, 381.34970605,
         383.34626251, 385.34281978, 387.33940104, 389.33597293,
         391.33254316, 393.32911301, 395.3256931 ]]))

In [128]:
a = testPredict-testY
a.mean()

-59.73402486840884

In [129]:
# testPredict += start
# testPredict

In [130]:
testPredict[11, 0]

325.32556

In [132]:
testPredict[10,0]

32442001000.0

In [138]:
start

167332273.50748467

In [133]:
testPredict[11,0]

32532556000.0

In [134]:
testPredict += start

In [135]:
testPredict *= (1 / 0.00000001)
testPredict

array([[3.1672062e+10],
       [3.1768470e+10],
       [3.1864265e+10],
       [3.1959458e+10],
       [3.2054041e+10],
       [3.2148042e+10],
       [3.2241455e+10],
       [3.2334285e+10],
       [3.2426541e+10],
       [3.2518220e+10],
       [3.2609333e+10],
       [3.2699888e+10],
       [3.2789879e+10],
       [3.2879323e+10],
       [3.2968221e+10]], dtype=float32)

creating my custom loss function, with a penalty for negative predictions

In [136]:
model_params = model.get_weights()